In [356]:
from torchvision import models
import numpy as np
import cv2
import PIL

from pytorch_grad_cam import GradCAM,GradCAMPlusPlus
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image,preprocess_image

import torch
import torchvision.transforms as transforms


In [357]:
def check_img(img):
        print(img.dtype)

        if np.min(img) < 0 or np.max(img) > 1:
                print("Values are not in the range [0, 1]")
                min_value = np.min(img)
                max_value = np.max(img)

                print("Minimum pixel value:", min_value)
                print("Maximum pixel value:", max_value)

        else:
                print("Image is valid") 

In [358]:
def predicted_class_acc(img, model):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),  # Resize to match ResNet's input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension

    # Feed the input tensor through the model
    with torch.no_grad():
        logits = model(input_tensor)

    # Apply softmax to get class probabilities
    probs = torch.softmax(logits, dim=1)

    # Get the predicted class label
    pred_class = torch.argmax(probs, dim=1).item()
    print("Predicted class:", pred_class)
    return pred_class

## Normal ResNet50

In [359]:
def normal_model(model, target_layers, targets, img):
    
    # instantiate the model
    cam = GradCAM(model=model, target_layers=target_layers) # use GradCamPlusPlus class

    input_tensor = preprocess_image(img)

    # generate CAM
    grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
    cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)

    cam = np.uint8(255*grayscale_cams[0, :])

    cam = cv2.merge([cam, cam, cam])

    # display the original image & the associated CAM
    images = np.hstack((np.uint8(255*img), cam_image))
    PIL.Image.fromarray(images)

    return cam_image, predicted_class_acc(img, model)



## Schizophrenia

In [360]:
def schizo_model(model, target_layers, targets, img):
    
    cam = GradCAM(model=model, target_layers=target_layers) # use GradCamPlusPlus class
    input_tensor = preprocess_image(img)

    # generate CAM
    grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
    cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)

    cam = np.uint8(255*grayscale_cams[0, :])

    cam = cv2.merge([cam, cam, cam])

    # display the original image & the associated CAM
    images = np.hstack((np.uint8(255*img), cam_image))
    PIL.Image.fromarray(images)

    return cam_image, predicted_class_acc(img, model)

## Gaussian Blur

In [361]:
def gauss_blur(model, target_layers, targets, img):

    cam = GradCAM(model=model, target_layers=target_layers) # use GradCamPlusPlus class
    # Define the location and size of the region to apply the blur
    x, y, w, h = (100, 100, 50, 50)  # Example: x and y are the coordinates of the top-left corner, w and h are the width and height

    # Apply Gaussian blur to the entire image
    blurred_img = cv2.GaussianBlur(img, (25, 25), 0)  # Adjust the kernel size for desired blur intensity

    # Create a mask for the square region (all zeros except for the square region which is filled with ones)
    mask = np.zeros_like(img)
    mask[y:y+h, x:x+w] = 255

    # Replace the blurred area with the original square region using the mask
    result_img = np.where(mask == 255, img, blurred_img)
    img_gauss = result_img

    input_tensor = preprocess_image(img_gauss)

    # generate CAM
    grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
    cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)

    cam = np.uint8(255*grayscale_cams[0, :])

    cam = cv2.merge([cam, cam, cam])

    # display the original image & the associated CAM
    images = np.hstack((np.uint8(255*img), cam_image))
    PIL.Image.fromarray(images)

    return cam_image, predicted_class_acc(img, model)    

In [362]:
# check_img(img_gauss)

## Color Augmentation

In [363]:
def aug_color(model, target_layers, targets, img):

    cam = GradCAM(model=model, target_layers=target_layers) # use GradCamPlusPlus class

    # # Split the image into its RGB channels
    b, g, r = cv2.split(img)

    # Decrease intensity of the red channel
    r = cv2.add(r, 0.4)  # Add 1 to the red channel values

    # Clip pixel values to ensure they stay within the range [0, 255]
    r = np.clip(r, 0, 1)

    # Merge the channels back together
    img_color = cv2.merge((b, g, r))

    input_tensor = preprocess_image(img_color)

    # generate CAM
    grayscale_cams = cam(input_tensor=input_tensor, targets=targets)

    cam_image = show_cam_on_image(img_color, grayscale_cams[0, :], use_rgb=True)
    cam = np.uint8(255*grayscale_cams[0, :])

    cam = cv2.merge([cam, cam, cam])

    # display the original image & the associated CAM
    images = np.hstack((np.uint8(255*img), np.uint8(255*img_color), cam_image))
    PIL.Image.fromarray(images)

    return cam_image, predicted_class_acc(img, model)    

In [364]:
# check_img(img_color)

In [365]:
## Normal model:

# use the pretrained ResNet50 model
model = models.resnet50(pretrained=True)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [366]:
## Schizophrenia model:

model_s = models.resnet50(pretrained=True)
model_s.eval()

# Modify the fully connected layer for your specific task
num_classes = 4  # Modify this based on your classification task
model_s.fc = torch.nn.Linear(2048, num_classes)

In [367]:
## targets: 

# fix the target layer (after which we'd like to generate the CAM)
target_layers = [model.layer4]
target_layers_s = [model_s.layer4]

# targets = [ClassifierOutputTarget(1)]

# targets = [ClassifierOutputTarget(208), ClassifierOutputTarget(155), ClassifierOutputTarget(254), ClassifierOutputTarget(235)]


In [368]:
names = ['Lab', 'ST', 'Pug', 'GS']
label_imagenet = [208, 208, 208, 155, 155, 155, 254, 254, 254, 235, 235, 235]
num_label = [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]
output_im_arr = np.array([])
predict_arr = []

for i in range(1): #, len(label_imagenet)):
    targets = [ClassifierOutputTarget(label_imagenet[i])] ## THIS IS THE ISSUE
    # targets = [ClassifierOutputTarget(1)]

    # print(f'Opening...', f'/Users/liviamurray/552_ALproj/dogs/{names[(i) // 3]}/{num_label[i]}.jpeg')
    img = np.array(PIL.Image.open(f'/Users/liviamurray/552_ALproj/dogs/{names[(i) // 3]}/{num_label[i]}.jpeg'))
    # img = np.array(PIL.Image.open(f'/Users/liviamurray/552_ALproj/goldfish-photo.jpg'))

    img = cv2.resize(img, (300,300))
    img = np.float32(img) / 255
    
    output_image, output_label = normal_model(model, target_layers, targets, img)
    cv2.imwrite(f'/Users/liviamurray/552_ALproj/output_images/{label_imagenet[i]}_{num_label[i]}_n2.jpg', output_image)
    # cv2.imwrite(f'/Users/liviamurray/552_ALproj/output_images/1_n.jpg', output_image)

    predict_arr.append([output_label, label_imagenet[i]])
        
    output_image, output_label = schizo_model(model_s, target_layers_s, targets, img)
    cv2.imwrite(f'/Users/liviamurray/552_ALproj/output_images/{label_imagenet[i]}_{num_label[i]}_s.jpg', output_image)
    # cv2.imwrite(f'/Users/liviamurray/552_ALproj/output_images/1_s.jpg', output_image)

    predict_arr.append([output_label, label_imagenet[i]])
        
    output_image, output_label = gauss_blur(model_s, target_layers_s, targets, img)
    cv2.imwrite(f'/Users/liviamurray/552_ALproj/output_images/{label_imagenet[i]}_{num_label[i]}_g.jpg', output_image)
    # cv2.imwrite(f'/Users/liviamurray/552_ALproj/output_images/1_g.jpg', output_image)

    predict_arr.append([output_label, label_imagenet[i]])
        
    output_image, output_label = aug_color(model_s, target_layers_s, targets, img)
    cv2.imwrite(f'/Users/liviamurray/552_ALproj/output_images/{label_imagenet[i]}_{num_label[i]}_c.jpg', output_image)
    # cv2.imwrite(f'/Users/liviamurray/552_ALproj/output_images/1_c.jpg', output_image)
    predict_arr.append([output_label, label_imagenet[i]])



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Predicted class: 208


IndexError: index 208 is out of bounds for dimension 0 with size 4

In [ ]:
predict_arr
corr = 0

for i, arr in enumerate(predict_arr):
    if arr[0] == arr[1]:
        corr += 1

print(f"Correctly predicted number: {corr} / {len(label_imagenet)}")

predict_arr

Correctly predicted number: 1 / 12


[[208, 208], [0, 208], [0, 208], [0, 208]]